In [1]:
#Import packages
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
#Import data
covid_diagnosis = pd.read_csv('Data/covid/diagnosis.csv',usecols =['patient_num','encounter_num',
                      'dx_date_shifted','dx_code'] , low_memory = False)
covid_encounters = pd.read_csv('Data/covid/encounters.csv',usecols = ['patient_num','encounter_num','admit_date_shifted',
                      'discharge_date_shifted','discharge_status_c', 'department_name'], low_memory = False)
df = pd.read_csv('Data/Opioid_1/diagnosis.csv',usecols =['patient_num','encounter_num',
                      'dx_date_shifted','dx_code'], low_memory = False)
df1 = pd.read_csv('Data/Opioid_2/diagnosis.csv',usecols =['patient_num','encounter_num',
                      'dx_date_shifted','dx_code'], low_memory = False)
opioid_diagnosis = pd.concat([df,df1], ignore_index=True)
df = pd.read_csv('Data/Opioid_1/encounters.csv',usecols = ['patient_num','encounter_num','admit_date_shifted',
                      'discharge_date_shifted','discharge_status_c', 'department_name'], low_memory = False)
df1 = pd.read_csv('Data/Opioid_2/encounters.csv',usecols = ['patient_num','encounter_num','admit_date_shifted',
                      'discharge_date_shifted','discharge_status_c', 'department_name'], low_memory = False)
opioid_encounters = pd.concat([df,df1], ignore_index=True)

In [3]:
#This function accepts diagnosis and encounters and then calculates the patients LOS and it also
#Specifys their diagnostic codes
def calc_los_dx_code(diagnosis,encounters):
    merge = diagnosis.merge(encounters,on=['patient_num','encounter_num'], how='left')
    merge.dropna(subset = ['admit_date_shifted','discharge_date_shifted','dx_date_shifted'],inplace = True)
    merge = merge.drop_duplicates(subset = 'patient_num', keep='first')
    merge.admit_date_shifted = pd.to_datetime(merge.admit_date_shifted)
    merge.discharge_date_shifted = pd.to_datetime(merge.discharge_date_shifted)
    merge.dx_date_shifted = pd.to_datetime(merge.dx_date_shifted)
    merge['admit_date_shifted'] = pd.to_datetime(merge['admit_date_shifted'], format='%Y-%m-%d')
    merge['discharge_date_shifted'] = pd.to_datetime(merge['discharge_date_shifted'], format='%Y-%m-%d')
    merge['admit_date_shifted'] = merge['admit_date_shifted'].dt.date
    merge['discharge_date_shifted'] = merge['discharge_date_shifted'].dt.date
    merge['length_of_stay_days'] = (merge['discharge_date_shifted'] - merge['admit_date_shifted']).dt.days
    merge.drop(['discharge_status_c', 'department_name'], axis=1, inplace=True)
    merge.dx_code = merge.dx_code.apply(str.split, sep=".").apply(list.__getitem__, args=(0,))
    col = np.append(arr = merge.dx_code.unique(), values ='LOS')
    return merge, col

In [4]:
#Call our function and then concatenate both tables to run through the pivot table function
#This will make sure we retain maximum information about the patients
#The col3 is the columns we will use for the pivot function
covid_data,col1 = calc_los_dx_code(covid_diagnosis,covid_encounters)
opioid_data,col2 = calc_los_dx_code(opioid_diagnosis,opioid_encounters)
total_data = pd.concat([covid_data,opioid_data],ignore_index = True)
col3 = np.unique(np.append(col1,col2))
col3 = np.append(col3,'admit_day_of_year')
col3 = np.append(col3,'admit_time_since_1970')
col3 = np.unique(col3)

In [5]:
#This function creates a pivot table from the entry style data and the columns you wish to include
def create_pivot_table(trim, col):
    df = pd.DataFrame(index=trim.patient_num.unique(), columns=col)
    for index, row in trim.iterrows():
        df.loc[row['patient_num'],row['dx_code']] = 1
        df.loc[row['patient_num'],'LOS'] = row['length_of_stay_days']
        df.loc[row['patient_num'],'admit_day_of_year'] = int(row['admit_date_shifted'].strftime('%j'))
        df.loc[row['patient_num'],'admit_time_since_1970'] = (row['admit_date_shifted'] - dt.date(1970,1,1)).days
    df = df.fillna(0)
    return df

In [6]:
#Create our pivot table, we then select only the covid patients add their true LOS and remove any
#features which are filled with zeros
total_table = create_pivot_table(total_data,col3)
table = total_table.loc[covid_data.patient_num.unique()]
for index, row in covid_data.iterrows():
    table.loc[row['patient_num'],'LOS'] = row['length_of_stay_days']
criteria = table.sum() >0
table = table[criteria.index[criteria]]
table.to_csv('Data/data_set/diagnosis.csv')